# Librerie utilizzate

In [8]:
from matplotlib import pyplot as plt

import matplotlib.dates as md
import numpy as np
from osgeo import gdal
from datetime import datetime
import time
import os
from os import listdir
from os.path import isfile, join,isdir
import rasterio
from rasterio import features,MemoryFile
from rasterio.enums import Resampling
import pprint
import rasterio.mask
import fiona

# Reorder scl files from directory

In [9]:
import os
from os import listdir
from os.path import isfile, join,isdir
import shutil

#Extract all SCL files from subdirectory and put them all under one folder
#original directory structure scl/year/month/day/0/R20m/SCL.tiff

path = join(os.path.abspath(os.getcwd()),"scl")
pathdest = join(os.path.abspath(os.getcwd()),"sclOrdered")
onlyfolders = [f for f in listdir(path) if (isdir(join(path, f)))]


for year in listdir(path):
    if (isdir(join(path, year))):
        for month in listdir(join(path,year)):
            for day in listdir(join(path,year,month)):
                file = join(path,year,month,day,"0","R20m","SCL.jp2")
                month1 = month
                day1 = day
                if(len(month1)<2):
                    month1 = "0"+month1
                
                if(len(day1)<2):
                    day1 = "0"+day1

                shutil.copy(file, join(pathdest,(year+"-"+month1+"-"+day1+".jp2")))
        
path = os.path.abspath(os.getcwd())
print(path)
print(onlyfolders)

c:\Users\Leo\Desktop\AgriVision2022
['2016', '2017']


# Scan and sort the name of files

In [10]:
img_path = join(os.path.abspath(os.getcwd()),"content","data")
scal_path = join(os.path.abspath(os.getcwd()),"sclOrdered")

img_list =[x.split('.')[0] for x in os.listdir(img_path)]
img_list.sort(key=lambda date: datetime.strptime(date, "%Y-%m-%d"))
print("Files and directories in '", img_path, "' :")
 
# prints all files
print(img_list)

print(img_list.index('2017-04-21'))
print(len(img_list))


Files and directories in ' c:\Users\Leo\Desktop\AgriVision2022\content\data ' :
['2016-11-05', '2016-11-12', '2016-11-15', '2016-11-22', '2016-11-25', '2016-12-02', '2016-12-05', '2016-12-12', '2016-12-15', '2016-12-22', '2016-12-25', '2017-01-01', '2017-01-04', '2017-01-14', '2017-01-24', '2017-01-31', '2017-02-03', '2017-02-10', '2017-02-13', '2017-02-20', '2017-02-23', '2017-03-02', '2017-03-05', '2017-03-12', '2017-03-15', '2017-03-22', '2017-03-25', '2017-04-01', '2017-04-04', '2017-04-11', '2017-04-14', '2017-04-21', '2017-04-24', '2017-05-01', '2017-05-04', '2017-05-11', '2017-05-14', '2017-05-21', '2017-05-24', '2017-05-31', '2017-06-03', '2017-06-10', '2017-06-13', '2017-06-20', '2017-06-23', '2017-06-30', '2017-07-03', '2017-07-08', '2017-07-10', '2017-07-13', '2017-07-15', '2017-07-18', '2017-07-20', '2017-07-23', '2017-07-25', '2017-07-28', '2017-07-30']
31
57


# Refill the transform

In [18]:

with rasterio.open(join(scal_path,img_list[30]+".jp2")) as referenceFile:
    TransformData = referenceFile.transform
CrsData = rasterio.open(join(scal_path,img_list[30]+".jp2")).crs

targetPath = join(os.path.abspath(os.getcwd()),"sclWriteTarget")
try:
    shutil.rmtree(targetPath)
    print("folder removed. proceeds to create a new folder.")
except:
    print("no folder to remove, proceeds to create a new folder.")
finally:
    os.makedirs(targetPath, exist_ok=True)

for img in range(31,57):

    dir = join(scal_path,img_list[img]+".jp2")
    scl = rasterio.open(dir)
    Data = scl.read(1)
    outSclMeta = scl.meta
    outSclMeta.update({"driver": "GTiff",
                 "height": Data.shape[0],
                 "width": Data.shape[1],
                 "transform": TransformData,
                 "crs":CrsData})
    scl.close()
    with rasterio.open(join(targetPath,img_list[img]+".jp2"), 'w', **outSclMeta) as dst:
        dst.write_band(1, Data)
    #re-copy to original destination
    shutil.copy(join(targetPath,img_list[img]+".jp2"), join(scal_path,(img_list[img]+".jp2")))

shutil.rmtree(targetPath)

no folder to remove, proceeds to create a new folder.
